# ALeRCE active galaxies starter notebook

```Author: Paula Sánchez-Sáez```

```Updated by Lorena Hernández-García, Last updated: 20221104```

ALeRCE starter notebook for active galaxy science using the ALeRCE client, APIs, and a direct database connection.

For more information about the ALeRCE broker, please visit http://alerce.science/, or read our publications:
* The Automatic Learning for the Rapid Classification of Events (ALeRCE) Alert Broker, [Förster et al. 2021, AJ, 161, 242](https://arxiv.org/abs/2008.03303)
* Alert Classification for the ALeRCE Broker System: The Real-time Stamp Classifier, [Carrasco-Davis et al. 2021, AJ, 162, 231](https://arxiv.org/abs/2008.03309)
* Alert Classification for the ALeRCE Broker System: The Light Curve Classifier, [Sánchez-Sáez et al. 2021, AJ, 161, 141](https://arxiv.org/abs/2008.03311)

Note that this notebook uses the ALeRCE Python client, which can be installed with pip (`pip install alerce`). For more information about this API and about the ALeRCE database visit http://alerce.science/.

We recommend running this tutorial from this
[Link](https://colab.research.google.com/github/alercebroker/usecases/blob/master/notebooks/ADASS_XXXII_Nov2022_tutorial/ALeRCE_AGN_starter.ipynb)

# Table of contents:
* [ALeRCE client](#client)
* [Requirements](#req)
* [Introduction](#intro)
* [Simple functions to plot light curves](#simplefunctions)
* [Advanced plotting functions including dust and distance attenuation corrections](#advancedfunctions)
* [Using Aladin](#aladdin)
* [Query many AGN and QSO candidates using the client](#agniaclient)
* [Query many AGN, QSO, and Blazar candidates using a direct DB connection](#agndb)
* [Query AGN, QSO, and Blazar features and PS1 photometry using a direct DB connection](#agndbother)

# ALeRCE client <a class="anchor" id="client"></a>

Install and import the alerce Python client


In [ ]:
#!pip install alerce

In [ ]:
from alerce.core import Alerce
client = Alerce()

# Requirements <a class="anchor" id="req"></a>

Basic requirements 

In [ ]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import requests

Astropy

In [ ]:
import astropy.units as u
from astropy import coordinates
from astropy.time import Time
from astropy.table import Table, Column
from astropy.coordinates import Distance
from astropy.cosmology import WMAP7

Astroquery https://astroquery.readthedocs.io/en/latest/

In [ ]:
#!pip install astroquery

from astroquery.irsa_dust import IrsaDust
from astroquery.ned import Ned



Other external packages

In [ ]:
from IPython.display import HTML
from ipywidgets import Layout, Box, widgets

# Introduction <a class="anchor" id="intro"></a>

### Classifiers and taxonomies

There are two classifiers used in ALeRCE ([Förster et al. 2021](https://arxiv.org/abs/2008.03303)):

1. A stamp based classifier ([Carrasco-Davis et al. 2021](https://arxiv.org/abs/2008.03309)), which classifies objects based on their first image stamps. The purpose of this classifier is to trigger alerts for young SN candidates and other objects.

    The stamp classifier contains the following classes:
    * `AGN`
    * `SN`
    * `VS`
    * `Asteroid`
    * `Bogus`


2. A light curve based classifier ([Sánchez-Sáez et al. 2021](https://ui.adsabs.harvard.edu/abs/2021AJ....161..141S/abstract)), which classifies objects based on their light curve. The purpose of this classifier is to provide a more refined classification starting with at least 6 detections in a given band.

    The light curve classifier contains the following classes:

    * `SNIa`
    * `SNIbc`
    * `SNII`
    * `SLSN`
    * `QSO`
    * `AGN`
    * `Blazar`
    * `CV/Nova`
    * `YSO`
    * `LPV`
    * `E`
    * `DSCT`
    * `RRL`
    * `CEP`
    * `Periodic-Other`

In this notebook we will focus on the results obtained by the light curve classifier, particularly for the case of active galaxies.

The ALeRCE light curve classifier ([Sánchez-Sáez et al. 2021](https://ui.adsabs.harvard.edu/abs/2021AJ....161..141S/abstract)) uses variability features computed from the ZTF alert stream, and colors obtained from AllWISE and ZTF photometry. It uses a Balanced Random Forest algorithm with a two-level scheme, where the top level classifies each source as periodic, stochastic, and transient, and the bottom level further resolve each hierarchical class, yielding a total of 15 classes. This classifier corresponds to the first attempt to classify multiple classes of stochastic variables (including nucleus- and host-dominated active galaxies, blazars, young stellar objects, and cataclysmic variables) in addition to different classes of periodic and transient sources, using real data.

The first version of the ALeRCE light curve classifier considers 15 subclasses of variable and transient objects, presented as a taxonomy tree defined by the ALeRCE collaboration:
![SanchezSaez+2020](https://github.com/alercebroker/usecases/blob/master/notebooks/figures/taxonomy_tree.png?raw=1)



In particular, the light curve classifier considers the following classes of active galaxies: 

* AGN: type 1 Seyfert galaxies (i.e., galaxies whose emission is dominated by the host galaxy), selected from MILLIQUAS (broad type "A"), and from Oh et al. 2015. 

* QSO: type 1 nucleus-dominated active galaxies (i.e., galaxies whose emission is dominated by their active nuclei), selected from MILLIQUAS (broad type "Q").

* Blazar: BL Lac objects and Flat Spectrum Radio Quasars (FSRQ), selected from ROMABZCAT and MILLIQUAS.

The confusion matrix from [Sánchez-Sáez et al. 2021](https://ui.adsabs.harvard.edu/abs/2021AJ....161..141S/abstract) shows that the light curve classifier has a very good performance separating QSOs, AGNs, and Blazars:

![SanchezSaez+2020](https://github.com/alercebroker/usecases/blob/master/notebooks/figures/SanchezSaez+2020_confusion.png?raw=1)

The recall curves as a function of magnitude and number of detections for active galaxies grouped as a single class are shown below:
<table><tr>
<td> <img src="https://github.com/alercebroker/usecases/blob/master/notebooks/figures/recall_agn_qso_blazar_for_mag.png?raw=1" alt="Drawing" style="width: 450px;"/> </td>
<td> <img src="https://github.com/alercebroker/usecases/blob/master/notebooks/figures/recall_agn_qso_blazar_for_ndet.png?raw=1" alt="Drawing" style="width: 450px;"/> </td>
</tr></table>

These recall curves are computed considering: a) the features available for the $g$ and $r$ bands (black triangles), b) the features available only for the $g$ band (green circles), and c) the features available only for the $r$ band (red squares). For details see [Sánchez-Sáez et al. 2021](https://ui.adsabs.harvard.edu/abs/2021AJ....161..141S/abstract). In general, both bands are available for the case of active galaxies. From these curves we can see that the light curve classifier has a very good performance selecting active galaxies as a single class. 

In this notebook we will show examples of how to use the different ALeRCE services for ative galaxy science. We will query data from a single AGN in order to plot its difference magnitude, corrected magnitude light curve, and data release (DR) light curve, and applying Milky Way dust attenuation corrections as well. 

Then, we will query many AGNs, QSOs, and Blazars using the ALeRCE client, showing some examples of light curves and stamps. Finally, we will repeat this query for a much larger sample size connecting directly to the ALeRCE database, and we will query and show some relevant features.


# Simple functions to plot light curves  <a class="anchor" id="simplefunctions"></a>


We will create three functions that plot the light curve given an object id (oid):

a) The first function (plot_diffLC) plots the difference magnitude light curve, as well as the non-detection diffmaglims (limiting magnitudes in the difference images). 

b) The second function (plot_corrLC) plots the apparent (or corrected) magnitude light curve (an estimate of the science images light curve). These corrected magnitudes are computed by the ALeRCE pipeline by summing up the flux measured in the difference images ($f_{\rm diff}$) and the flux measured from the template image ($f_{\rm ref}$), taking into account whether the difference between the science and reference image in a given epoch ($\rm sgn$) is positive or negative (for more details see the appendix of Förster et al. 2021) :

$$
    \hat f_{\rm sci} = f_{\rm ref} + {\rm sgn}~ f_{\rm diff}
$$

c) The third function (plot_corrLC_DR) plots the apparent (or corrected) magnitude light curve using the alerts and the data release (DR). While alerts are generated for changes above 5$\sigma$ respect to a reference image, the DR performs the photometry measurements on every single science image. This results in a complete light curve since 2018. ZTF releases the DR every two months.


These functions use a dataframe with detections and a dataframe with non detections.

We will first define manually the oid which we want to explore (we select an AGN with known redshift from NED for test purposes, we will later do queries for a collection of sources).

In [ ]:
seloid = "ZTF18aajupnt" # changing-look LINER from Frederick et al. 2019 
                        # (https://ui.adsabs.harvard.edu/abs/2019ApJ...883...31F/abstract)

We start defining the function that plots the difference image magnitudes

In [ ]:
def plot_diffLC(oid, LC_det, LC_nondet): 
    
    fig, ax = plt.subplots(figsize = (14, 8))
    labels = {1: 'g', 2: 'r'}
    colors = {1: '#56E03A', 2: '#D42F4B'}
    markers = {1: 'o', 2: 's'}
    
    # loop the passbands
    for fid in [1, 2]:
        
        # plot detections if available
        mask = LC_det.fid == fid
        if np.sum(mask) > 0:
            # note that the detections index is candid and that we are plotting the psf corrected magnitudes
            ax.errorbar(LC_det[mask].mjd, LC_det[mask].magpsf, 
                yerr = LC_det[mask].sigmapsf, c = colors[fid], fmt=markers[fid], label = labels[fid])
        
        # plot non detections if available and if wanted:
        mask = (LC_nondet.fid == fid) & (LC_nondet.diffmaglim > -900)
        if (np.sum(mask) > 0):     
            # non detections index is mjd
            ax.scatter(LC_nondet[mask].mjd, LC_nondet[mask].diffmaglim, c = colors[fid], alpha = 0.5,
                marker = 'v', label = "lim.mag. %s" % labels[fid])
            
    ax.set_title(oid)
    ax.set_xlabel("MJD")
    ax.set_ylabel("difference magnitude")
    ax.legend()
    ax.set_ylim(ax.get_ylim()[::-1])

And now the function that plot the corrected magnitude light curve

In [ ]:
def plot_corrLC(oid, LC_det):
    
    fig, ax = plt.subplots(figsize = (14, 8))
    labels = {1: 'g', 2: 'r'}
    colors = {1: '#56E03A', 2: '#D42F4B'}
    markers = {1: 'o', 2: 's'}
    
    # loop the passbands
    for fid in [1, 2]:
        
        # plot detections if available
        mask = LC_det.fid == fid
        if np.sum(mask) > 0:
            # note that the detections index is candid and that we are plotting the psf corrected magnitudes
            ax.errorbar(LC_det[mask].mjd, LC_det[mask].magpsf_corr, 
                yerr = LC_det[mask].sigmapsf_corr_ext, c = colors[fid], fmt=markers[fid], label = labels[fid])
        
            
    ax.set_title(oid)
    ax.set_xlabel("MJD")
    ax.set_ylabel("Apparent magnitude")
    ax.legend()
    ax.set_ylim(ax.get_ylim()[::-1])

We now create a function to get data from the ZTF Data Release (DR).  As a result, we will have the data of the source from 2018, independently of the alerts. 

In [ ]:
def get_DR(oid, url="https://api.alerce.online/ztf/dr/v1/light_curve/"):
    stats = client.query_object(oid, format='pandas')
    
    ra = float(stats.meanra)
    dec = float(stats.meandec)
    query = {'ra':ra, 'dec':dec, 'radius':1.5}

    response = requests.get(url, query)
    output = response.json()

    df = []
    for i in output:
        aux = pd.DataFrame({'hmjd': np.array(i["hmjd"]), 'mag': np.array(i["mag"]), "magerr": np.array(i["magerr"])})
        aux["ID"] = int(i["_id"])
        aux["filterid"] = int(i["filterid"])
        aux["oid"] = oid
        df.append(aux)

    if df != []:  
        return pd.concat(df) #output
    else:
        return None

In [ ]:
# get candidate ZTF data release information
DR = get_DR(seloid)
#print(DR.columns)

And then we define the function to plot the ZTF DR light curve in combination with the alerts light curve for comparison:

In [ ]:
def plot_corrLC_DR(oid, LC_det, DR):
    
    fig, ax = plt.subplots(figsize = (14, 8))
    labels = {1: 'g', 2: 'r'}
    labels1 = {1: 'g DR', 2: 'r DR'}
    colors = {1: '#56E03A', 2: '#D42F4B'}
    colors1 = {1: '#ADA3A3', 2: '#377EB8',}
    markers = {1: 'o', 2: 's'}
    
    # loop the passbands
    for fid in [1, 2]:
        
        # plot detections if available
        mask = LC_det.fid == fid
        maskDR = DR.filterid == fid
        if np.sum(mask) > 0:
            # note that the detections index is candid and that we are plotting the psf corrected magnitudes
            ax.errorbar(DR[maskDR].hmjd, DR[maskDR].mag, 
                yerr = DR[maskDR].magerr, c = colors1[fid], fmt=markers[fid], label = labels1[fid], alpha=.3)
            ax.errorbar(LC_det[mask].mjd, LC_det[mask].magpsf_corr, 
                yerr = LC_det[mask].sigmapsf_corr_ext, c = colors[fid], fmt=markers[fid], label = labels[fid])
            
        
            
    ax.set_title(oid)
    ax.set_xlabel("MJD")
    ax.set_ylabel("Apparent magnitude")
    ax.legend()
    ax.set_ylim(ax.get_ylim()[::-1])

Now, we will create a function that gets the light curve data and plots it if required, selecting whether we want the difference, the corrected magnitude, or the DR light curves. The results are then returned to the user as a dictionary. Note that we use the client to query the detections and non detections and that we return the results in pandas format (default is votable).

In [ ]:
def getLCdata(oid, doLC = False,  plot_diff = False, doDR = False):
    
    # show link
    display(HTML("<a href='http://alerce.online/object/%s' target=\"_blank\"> %s <a>" % (oid, oid)))

    results = {"oid": oid}
        
    # query detections
    LC_det = client.query_detections(oid, format='pandas')
    results["lc_det"] = LC_det
        
    # query non detections
    LC_nondet = client.query_non_detections(oid, format='pandas')
    results["lc_nondet"] = LC_nondet
    
    # plot the LC
    if doLC:
        if plot_diff:
            plot_diffLC(oid, LC_det, LC_nondet)
        else: 
            plot_corrLC(oid, LC_det)
            
    # plot the LC
    if doDR:
        if plot_diff:
            plot_diffLC(oid, LC_det, LC_nondet)
        else: 
            plot_corrLC_DR(oid, LC_det, DR)
        
    # return data
    return results

And now we can plot the object's difference light curve, including detections and non-detections

In [ ]:
results = getLCdata(seloid, doLC=True, plot_diff = True)

And the object's corrected light curve, showing only the detections

In [ ]:
results = getLCdata(seloid, doLC=True)

As well as the ZTF DR light curve, along with the alerts light curve:

In [ ]:
results = getLCdata(seloid, doDR = True)

ZTF18aajupnt (AT2018dyk) was classified as a changing-look LINER by [Frederick et al. 2019](https://ui.adsabs.harvard.edu/abs/2019ApJ...883...31F/abstract). This source was first [classified as TDE](https://wis-tns.weizmann.ac.il/object/2018dyk/classification-cert), however its light curve does not show the power-law decline characteristic of an optical TDE. Frederick et al. 2019 performed a detailed multi-wavelength photometric and spectroscopic analysis of this target (see Section 3.6), and demonstrated that ZTF18aajupnt is an "extreme" changing-look LINER, which transitioned into a narrow-line Seyfert 1 (NLS1).

ZTF18aajupnt is classified as AGN by the light curve classifier, with a probability of 0.65.

If we compare the alerts light curve with the DR light curve, we can see the difference in dates, with alerts up to 58700 MJD, and the DR up to 59500 MJD.

Note the difference between the difference and corrected light curves. The corrected light curve of ZTF18aajupnt is much redder. 




It is important to use the corrected light curve for variability analyses, in particular for sources with changes in the sign of the difference between the reference and the science image. This can be seen in the light curve of Mrk 817 (ZTF18aamgjku):

In [ ]:
seloid = "ZTF18aamgjku"
results = getLCdata(seloid, doLC=True, plot_diff = True)

In [ ]:
results = getLCdata(seloid, doLC=True)

In [ ]:
DR = get_DR(seloid)
results = getLCdata(seloid, doDR=True)

This target (Mrk 817) is particularly interesting, since it presents a change in the sign of the difference between the reference and the science image, therefore the real shape of the light curve cannot been clearly distinguished using from the difference magnitude light curve. That is why is very important for the study of active galaxies to always use the apparent or corrected magnitude light curve. Hereafter we will use the corrected light curves. 

 

# Advanced plotting functions including dust and distance attenuation corrections  <a class="anchor" id="advancedfunctions"></a>

We will modify our previous function (plot_corrLC) and now we will be able to receive a dictionary with attenuations due to Galactic extinction, as well as the redshift. This will allow us to get absolute magnitudes (assuming no K-corrections), and to 
transform the light curve to the source rest frame. Note that we use astroquery Distance and WMAP7 objects, as well as mag units.

In [ ]:
def plotLC(oid, LC_det, A=None, redshift=None):
    
    fig, ax = plt.subplots(figsize = (14, 8))
    labels = {1: 'g', 2: 'r'}
    colors = {1: '#56E03A', 2: '#D42F4B'}
    markers = {1: 'o', 2: 's'}
    
    # distmod: distance modulus, reftime: reference time
    if redshift is not None:
        distmod = float(Distance(z=redshift, cosmology=WMAP7).distmod / u.mag)
        reftime = LC_det.mjd.min()
    else:
        distmod = 0
        reftime = 0
        redshift = 0
        
    # loop the passbands
    for fid in [1, 2]:
        
        # galactic extinction in the given band
        if A is not None:
            A_fid = A[labels[fid]]
        else:
            A_fid = 0

        # plot detections if available
        mask = LC_det.fid == fid
        if np.sum(mask) > 0:
            
            # change time to restframe days if redshift is not zero
            times = (LC_det[mask].mjd - reftime) / (1. + redshift)
            
            # note that the detections index is candid and that we are plotting the psf corrected magnitudes
            if A_fid != 0:
                # plot attenuated light curve with small alpha
                ax.errorbar(times, LC_det[mask].magpsf_corr - distmod, 
                    yerr = LC_det[mask].sigmapsf_corr_ext, c = colors[fid], fmt=markers[fid], label = labels[fid], alpha=0.1)
                # show dust and distance attenuation corrected light curve
                ax.errorbar(times, LC_det[mask].magpsf_corr - A_fid - distmod, 
                    yerr = LC_det[mask].sigmapsf_corr_ext, c = colors[fid], fmt=markers[fid], label = "%s (ext. corr.)" % labels[fid])
            else:
                # correct only for redshift
                ax.errorbar(times, LC_det[mask].magpsf_corr - distmod, 
                    yerr = LC_det[mask].sigmapsf_corr_ext - A_fid, c = colors[fid], fmt=markers[fid], label = labels[fid])
        
            
    # labels
    if A is not None and redshift != 0:
        ax.set_title("%s ($A_g=%.3f, A_r=%.3f$, z=%.4f)" % (oid, A["g"], A["r"], redshift))
    elif redshift != 0:
        ax.set_title("%s (z=%.4f)" % (oid, redshift))
    elif A is not None:
        ax.set_title("%s ($A_g=%.3f, A_r=%.3f$)" % (oid, A["g"], A["r"]))
    else:
        ax.set_title(oid)
        
    if redshift == 0:
        ax.set_xlabel("MJD")
        ax.set_ylabel("Apparent magnitude")
    else:
        ax.set_xlabel("Restframe time [days]")
        ax.set_ylabel("Absolute magnitude (no K-corr.)")
    ax.legend()
    ax.set_ylim(ax.get_ylim()[::-1])

Now we test these functions with the same AGN as before (Mrk 817). This source has a known spectrocopic redshift, but this is usually not the case.

We will also modify the getLCdata function to get all the necessary information given the object id (oid). It now does the following:
 - display a link to the ALeRCE explorer page for this object
 - get the basic object statistics, this includes the position in the sky (using the ALeRCE client)
 - get the detections (using the ALeRCE client). Non-detections are excluded now.
 - get the Milky Way dust attenuation at the given position in the sky (using astroquery IrsaDust)
 - plot the apparent magnitude light curve corrected for dust attenuation (using the previously defined function)
 - query NED to get the redshift, using astroquery Ned, and a radius of 2 arcseconds
 - plot the absolute magnitude light curve correction for Milky Way dust and distance attenuation, in restframe time and ignoring k-corrections (using the previously defined plotLC function)
 - plot the first image stamps associated to this source (using the ALeRCE client)

In [ ]:
def getLCdata(oid, doLC=False, doext=False, dostamp=False, doNED=False, doredshift=False):

    # show link
    display(HTML("<a href='http://alerce.online/object/%s' target=\"_blank\"> %s <a>" % (oid, oid)))
    
    results = {"oid": oid}
    
    # query basic stats
    LC_stats = client.query_object(oid, format='pandas')
    results["stats"] = LC_stats
    
    # query detections
    LC_det = client.query_detections(oid, format='pandas')
    results["lc_det"] = LC_det
    
    # object coordinates
    co = coordinates.SkyCoord(ra=LC_stats.meanra[0], dec=LC_stats.meandec[0], unit=(u.deg, u.deg), frame='fk5')
    
    # get galactic extinction
    if doext:
        # Galactic extinction (use SFD = D.J. Schlegel, D.P. Finkbeiner, & M. Davis (1998, ApJ, 500, 525))
        table = IrsaDust.get_extinction_table(co)
        A = {"g": float(table[table["Filter_name"] == "SDSS g"]["A_SFD"]), "r": float(table[table["Filter_name"] == "SDSS r"]["A_SFD"])}
        results["attenuation"] = A

    # plot the LC
    if doLC:
        if doext:
            plotLC(oid, LC_det, A)
        else:
            plotLC(oid, LC_det)
            
    # find NED galaxies
    if doNED:
        Ned_table = Ned.query_region(co, radius=2.0 * u.arcsec, equinox='J2000.0')
        display(Ned_table)
        results["ned_table"] = Ned_table
        
        # save closest redshift if any
        df_ned = Ned_table.to_pandas()
        mask=df_ned.Redshift.notna()
        if mask.sum() > 0:
            results["redshift"] = df_ned.loc[df_ned.loc[mask].Separation.idxmin()].Redshift
            print("Redshift found: %s" % results["redshift"])
            
    # plot absolute value (without K-corrections)
    if doredshift:
        if type(doredshift) == float and doredshift > 0:
            results["redshift"] = doredshift
        if "redshift" in results.keys():
            if doext:
                plotLC(oid, LC_det, A, results["redshift"])
            else:
                plotLC(oid, LC_det, results["redshift"])            
    
    
    
    
    # show the first image stamp
    if dostamp:
        candid = results["lc_det"].loc[results["lc_det"].has_stamp].candid.min()
        stamps = client.get_stamps(oid, candid)
        science, ref, difference = stamps[0].data, stamps[1].data, stamps[2].data
        fig, ax = plt.subplots(ncols=3, figsize=(12, 6))
        for idx, im in enumerate([np.log(science), np.log(ref), difference]):
            ax[idx].imshow(im, cmap='viridis') # Log scale for visualization
            ax[idx].axes.get_xaxis().set_visible(False)
            ax[idx].axes.get_yaxis().set_visible(False)
        ax[0].set_title("oid: %s, candid: %s (science, reference and difference)" % (oid, candid), loc='left')
        fig.subplots_adjust(wspace = 0, hspace = 0)
        
    # return data
    return results

In [ ]:
results_1st_target = getLCdata(seloid, doLC=True, doext=True, dostamp=True, doNED=True, doredshift=True)

This source has the typical absolute magnitude observed for low redshift active galaxies. 

You can also force the absolute magnitude plot by giving the redshift manually. This may be necessary if there is not a redshift in NED or you need to use a much larger search radius to find it automatically. In this case we selected a source classified as GALAXY by the DR16 catalog "DR16Q_Superset_v3" ([Lyke et al. 2020]( https://ui.adsabs.harvard.edu/abs/2020arXiv200709001L/abstract)). 

In [ ]:
results_2nd_target = getLCdata("ZTF18abotama", doLC=True, doext=True, dostamp=True, doNED=True, doredshift=0.3471357);

It can be seen that this source shows a variable flux, but its [SDSS spectra](http://skyserver.sdss.org/dr15/en/tools/chart/navi.aspx?ra=24.746433972028&dec=26.0043609167832) shows no clear signal of SMBH activity 

# Using Aladin   <a class="anchor" id="aladdin"></a>

We can show an image from Aladin to see the source and its environment (we could also use ipyaladin for more interactivity, but it is not currently working in Colab). In this case we continue working with Mrk 817.

In [ ]:
# Use the example code from Aladin Lite
def show_image(results):
  query = """
<!-- include Aladin Lite CSS file in the head section of your page -->
<link rel="stylesheet" href="https://aladin.u-strasbg.fr/AladinLite/api/v2/latest/aladin.min.css" />
 
<!-- you can skip the following line if your page already integrates the jQuery library -->
<script type="text/javascript" src="https://code.jquery.com/jquery-1.12.1.min.js" charset="utf-8"></script>
 
<!-- insert this snippet where you want Aladin Lite viewer to appear and after the loading of jQuery -->
<div id="aladin-lite-div" style="width:800px;height:400px;"></div>
<script type="text/javascript" src="https://aladin.u-strasbg.fr/AladinLite/api/v2/latest/aladin.min.js" charset="utf-8"></script>
<script type="text/javascript">
    var aladin = A.aladin('#aladin-lite-div', {survey: "P/PanSTARRS/DR1/color-z-zg-g", fov:0.04, target: "%s, %s"});
</script>
""" % (float(results["stats"].meanra), float(results["stats"].meandec))
  display(HTML(query))

In [ ]:
# this uses the results from the last getSNdata call
show_image(results_1st_target)

# Query many AGN and QSO candidates using the client <a class="anchor" id="agniaclient"></a>

The ALeRCE client allows users to access the ALeRCE API and do simple queries to our database. You can get information about each command using the help command.

In [ ]:
help(client.query_objects)

### Querying the ALeRCE database

We will now query the top 1000 AGN and top 1000 QSO candidates, that have a probability > 0.6 of being a AGN according to the light curve classifier, and which have between 30 and 200 detections.

In [ ]:
min_firstmjd = Time("2017-11-01T00:00:00", format="isot", scale="utc").mjd

AGNs = client.query_objects(classifier="lc_classifier",
                           class_name="AGN", 
                           probability=0.6,
                           ndet=[30, 200],
                           order_by="probability",
                           order_mode="DESC",
                           first_mjd=[min_firstmjd, None],
                           count=False,
                           page_size=1000, format='pandas')
print(AGNs.shape)
AGNs.set_index("oid", inplace=True)
AGNs.head()

In [ ]:
QSOs = client.query_objects(classifier="lc_classifier",
                           class_name="QSO", 
                           probability=0.6,
                           ndet=[30, 200],
                           order_by="probability",
                           order_mode="DESC",
                           first_mjd=[min_firstmjd, None],
                           count=False,
                           page_size=1000, format='pandas')
print(QSOs.shape)
QSOs.set_index("oid", inplace=True)
QSOs.head()

Here you can see the fields available in the dataframe

In [ ]:
", ".join(list(AGNs))

This table contains the unique object identifier `oid` as index, the classification class `class`, whether its light curve was corrected `corrected`, the light curve length `deltjd`, the time of first detection `firstmjd`, the g-r color at maximum `g_r_max`, the corrected g-r color at maximum `g_r_max_corr`, the mean g-r color `g_r_mean`, the corrected mean g-r color `g_r_mean_corr`, the time of last detection `lastmjd`, the mean declination `meandec`, the mean right ascension `meanra`, the latest time of a raw SNR$\sim 3$ detection `mjdendhist`, the earliest time of a raw SNR$\sim 3$ detection `mjdstarthist`, the number of times the candidate fell inside on a ZTF observation `ncovhist`, the number of detections `ndet`, the number of raw SNR$\sim 3$ detection `ndethist`, the classification probability `probability`, the declination standard deviation `sigmadec`, the right ascension standard deviation `sigmara`, whether the object is likely stellar `stellar`, and the version of the preprocessing step `step_id_corr`.

Now we show the corrected mean g-r color `g_r_mean_corr` distribution of AGNs and QSOs:

In [ ]:
fig, ax = plt.subplots()
bins = np.linspace(-0.3,1.5,30)
ax.hist(AGNs.g_r_mean_corr,bins = bins, lw=2, histtype='step', color = "firebrick", label = "AGN candidates")
ax.hist(QSOs.g_r_mean_corr, bins=bins, lw=2, histtype='step', color = "royalblue", label = "QSO candidates")
ax.set_xlabel(r"($g-r$)_mean_corr")
plt.legend()

And the mean g-r color `g_r_mean` (i.e., measured from the difference images) distribution of AGNs and QSOs:

In [ ]:
fig, ax = plt.subplots()
bins = np.linspace(-0.3,1.5,30)
ax.hist(AGNs.g_r_mean,bins = bins, lw=2, histtype='step', color = "firebrick", label = "AGN candidates")
ax.hist(QSOs.g_r_mean, bins=bins, lw=2, histtype='step', color = "royalblue", label = "QSO candidates")
ax.set_xlabel(r"($g-r$)_mean_corr")
plt.legend()

Clearly AGNs are much redder than QSOs when we use the corrected magnitudes, while when we consider the difference magnitudes their color distributions are not so different.

Finally, we try some random AGNs from the selected sample. Some of them will have a known redshift.

In [ ]:
agn_sel = np.random.choice(AGNs.index, size=5)

In [ ]:
results = getLCdata(agn_sel[0], doLC=True, doext=True, dostamp=True, doNED=True, doredshift=True)


In [ ]:
results = getLCdata(agn_sel[1], doLC=True, doext=True, dostamp=True, doNED=True, doredshift=True)


# Query many AGN, QSO, and Blazar candidates using a direct DB connection <a class="anchor" id="agndb"></a>

The previous way of querying candidates is good for relatively simple queries. However, if we want to do more complex queries the best solution may be to connect directly to the DB. We show how to do this here.

First, you need to install and import pyscopg2

In [ ]:
#!pip install psycopg2-binary
import psycopg2

Then, you will need to use the read only credentials available in the repository

In [ ]:
import json
import requests
url = "https://raw.githubusercontent.com/alercebroker/usecases/master/alercereaduser_v4.json"
params = requests.get(url).json()['params']

Now we open a connection to the DB

In [ ]:
conn = psycopg2.connect(dbname=params['dbname'], user=params['user'], host=params['host'], password=params['password'])

And we show all the available tables

In [ ]:
query = """
SELECT table_name  FROM information_schema.tables
WHERE table_schema='alerce'
ORDER BY table_name;
"""
tables = pd.read_sql_query(query, conn)
tables.sort_values(by="table_name")

You can see all the tables used in the new version of our database. The most relevant tables are, moving from less to more aggregation:

* `non_detection`: one row per non-detection per object, the limiting magnitudes
* `detection`: one row per detection, light curves and other relevant time dependent information
* `data_quality`: one row per detection, data quality related time dependent information
* `magstat`: one row per object per filter, statistics per bandpass per object
* `object`: one row per object, basic object statistics
* `probability`: one row per object per classifier and class, the probabilities of every object
* `reference`: one row per object per reference image, object statistics for every reference image used
* `feature`: one row per object per feature, object computed features
* `xmatch`: one row per object per external catalog, the table that points to the detailed xmatch tables
* `allwise, ps1_ztf, gaia_ztf, ss_stf`: one row per object, xmatch tables

For completeness, we now show all columns available in all tables!

In [ ]:
alltabs = []
for tab in sorted(tables.table_name):
    cols = pd.DataFrame()
    query = "select column_name, data_type from information_schema.columns where table_name = '%s';" % tab
    results = pd.read_sql_query(query, conn)
    results["table"] = tab
    alltabs.append(results)
dftab = pd.concat(alltabs)
pd.options.display.max_rows = 999
display(dftab[["table", "column_name", "data_type"]])
pd.options.display.max_rows = 101

Now we can do a query asking for sources classfied as AGN, QSO or Blazar. We define a minimum number of detections (ndet), and a minimum time span between the first and the last detection (deltajd), in order to ensure the detection of a variable signal from the light curves.  We also filter out sources by their predicted class probability. The DB includes a larger number of QSO and AGN than Blazars, so to be able to collect similar numbers, we include a UNION in this query. WARNING: this might take some time to run depending on the selection criteria...


In [ ]:
query='''
SELECT
    object.oid, object.meanra, object.meandec, object.ndet,
    object.firstMJD, object.deltajd, object.g_r_mean_corr,
    probability_oid.classifier_name, probability_oid.class_name,
    probability_oid.ranking, probability_oid.probability
FROM
    object INNER JOIN (
    SELECT
        oid, classifier_name, class_name, ranking, probability
    FROM
        probability
    WHERE
        classifier_name='lc_classifier'
        AND class_name IN ('AGN', 'QSO')
        AND ranking=1
        AND probability > 0.95
    UNION
    SELECT
        oid, classifier_name, class_name, ranking, probability
    FROM
        probability
    WHERE
        classifier_name='lc_classifier'
        AND class_name IN ('Blazar')
        AND ranking=1
        AND probability > 0.8
    ) AS probability_oid
    ON object.oid=probability_oid.oid
WHERE
    probability_oid.classifier_name='lc_classifier'
    AND object.deltajd > 200
    AND object.ndet > 40
'''

objects = pd.read_sql_query(query, conn)
print(objects.shape)
objects.set_index('oid', inplace=True)
objects.head()

How many sources are classified as QSOs, AGNs or Blazars?

In [ ]:
objects.class_name.value_counts().plot.barh()

print("QSO: ", len(objects.loc[objects["class_name"]=="QSO"]))
print("AGN: ", len(objects.loc[objects["class_name"]=="AGN"]))
print("Blazar: ", len(objects.loc[objects["class_name"]=="Blazar"]))

We save the object IDs (oid) of each source, which will be used in the next sections:

In [ ]:
oids_objects =  objects.index.values
#print(len(oids_objects))
#print(oids_objects)

# Query AGN, QSO, and Blazar features and PS1 photometry using a direct DB connection <a class="anchor" id="agndbother"></a>

In order to understand the differences between the QSO, AGN and Blazar candidates, we search for its Pan-STARRS1 photometry, using the ps1_ztf table. WARNING: this might take some time to run...

In [ ]:
query='''
SELECT
    oid, sgmag1, srmag1, simag1, szmag1, sgscore1
FROM
    ps1_ztf
WHERE
    oid in (%s)
''' % ",".join(["'%s'" % oid for oid in oids_objects])
ps1 = pd.read_sql_query(query, conn)
ps1.set_index("oid", inplace=True)
ps1.head()

We use this data to plot a color-color diagram of QSO, AGN and Blazar candidates

In [ ]:
objects["g_r_ps1"] = ps1.sgmag1 - ps1.srmag1
objects["r_i_ps1"] = ps1.srmag1 - ps1.simag1
objects["i_z_ps1"] = ps1.simag1 - ps1.szmag1
objects["sgscore1"] = ps1.sgscore1 
objects.head()

In [ ]:
fig, ax = plt.subplots(figsize = (8,8))  

for class_name, color in [("QSO","lightsteelblue"),("AGN","lightcoral"),("Blazar","olivedrab")]:
  plt.scatter(objects.loc[objects["class_name"]==class_name,"g_r_ps1"].values,objects.loc[objects["class_name"]==class_name,"r_i_ps1"].values,c=color,s=12,label=class_name)

plt.ylabel(r'$r-i$',fontsize=20)
plt.xlabel(r'$g-r$',fontsize=20)
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
plt.locator_params(axis='y', nbins=6)
plt.locator_params(axis='x', nbins=6)
plt.legend(fontsize = 15,markerscale=5)
plt.ylim(-0.8,1.3)
plt.xlim(-0.6,1.6)

This color-color diagram shows that QSO candidates are bluer than AGN and Blazar candidates.

The sgscore1 corresponds to a morphological star/galaxy score of the closest source from PanSTARRS1 ([Tachibana et al. 2018](https://ui.adsabs.harvard.edu/abs/2018PASP..130l8001T/abstract)), with $0 \leq \texttt{sgscore1} \leq 1$, where values closer to 1 imply a higher likelihood of the source being a star. Let's see the distribution of the sgscore1 parameter for these sources:

In [ ]:
fig, ax = plt.subplots(figsize = (8,8)) 
bins_hist = np.linspace(0,1,25,endpoint=True)

for class_name, color in [("QSO","royalblue"),("AGN","firebrick"),("Blazar","olivedrab")]:
  weights = np.ones_like(objects.loc[objects["class_name"]==class_name,"sgscore1"].values)/float(len(objects.loc[objects["class_name"]==class_name,"sgscore1"].values))
  plt.hist(objects.loc[objects["class_name"]==class_name,"sgscore1"].values, weights=weights, cumulative=False, stacked=False, density=False, histtype='step', bins=bins_hist, lw=3, color = color, ec=color,label=class_name, alpha=1.0)
  
plt.legend(loc='upper left',fontsize=17)
plt.xlabel(r'sgscore1',fontsize=20)
plt.ylabel(r'normalized frequency',fontsize=20)
plt.xticks(fontsize = 17)
plt.yticks(fontsize = 17)


Clearly AGN candidates have more extended morphologies compared to QSOs and Blazars.


In addition, we query some relevant features for active galaxies from the feature table, like IAR_phi, GP_DRW_sigma and GP_DRW_tau, the MHPS feautres, ExcessVar, and the Structure Function features. These features are described in detail in Sánchez-Sáez et al. 2021. Note that these features are computed using the observed frame light curves (i.e., the light curves are not corrected by redshift). WARNING: this might take some time to run...

In [ ]:
query='''
SELECT
    *
FROM
    feature
WHERE
     oid in (%s)
''' % ",".join(["'%s'" % oid for oid in oids_objects])


features = pd.read_sql_query(query, conn)
features.head()

Let's save these features in a new dataframe to make their manipulation easier 

In [ ]:
features["fname"]=features.name + "_" + features.fid.astype(str)
features_aux = features.pivot_table(index="oid", columns="fname",values="value")
features_aux.head()

In [ ]:
objects = objects.join(features_aux)
objects.head()

Let's see the distribution of some of these features:

First, we will plot the distribution of the Mexican Hat Power Spectrum (MHPS) features. These features are based on the method presented in [Arévalo et al. 2012](https://ui.adsabs.harvard.edu/abs/2012MNRAS.426.1793A/abstract). They proposed a method to compute low-resolution power spectra from data with gaps, where the light curves are convolved with a Mexican hat filter. This method can be used to isolate structures  with a characteristic timescale in a given light curve, in order to estimate the light curve normalized variance associated with that timescale. We compute the light curve variance at two different timescales of 10 (high frequency; MHPS\_high) and 100 days (low frequency; MHPS\_low). We also compute the ratio between the low and high frequency variances for a given band, denoted as MHPS\_ratio. The logarithm of MHPS\_ratio is therefore an estimate of the power law slope of the power spectrum of the light curve.

Below we plot the distribution of these features for the $g$ (\_1) and $r$ band (\_2)

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols= 3, figsize = (18,10)) 

for feature, axr,axc in [("MHPS_low_1",0,0),("MHPS_high_1",0,1),("MHPS_ratio_1",0,2),("MHPS_low_2",1,0),("MHPS_high_2",1,1),("MHPS_ratio_1",1,2)]:
  if feature=="MHPS_ratio_1" or feature=="MHPS_ratio_2": bins_hist = np.logspace(np.log10(1e-1),np.log10(1500),25, base=10.0,endpoint=True)
  else: bins_hist = np.logspace(np.log10(1e-5),np.log10(1300),25, base=10.0,endpoint=True)
  
  objects_aux = objects[objects[feature]>=np.min(bins_hist)]
  for class_name, color in [("QSO","royalblue"),("AGN","firebrick"),("Blazar","olivedrab")]:
    weights = np.ones_like(objects_aux.loc[objects_aux["class_name"]==class_name,feature].values)/float(len(objects_aux.loc[objects_aux["class_name"]==class_name,feature].values))
    ax[axr,axc].hist(objects_aux.loc[objects_aux["class_name"]==class_name,feature].values, weights=weights, cumulative=False, stacked=False, density=False, histtype='step', bins=bins_hist, lw=3, color = color, ec=color,label=class_name, alpha=1.0)

  ax[axr,axc].set_xlabel(feature,fontsize=14)
  ax[axr,axc].set_ylabel(r'normalized frequency',fontsize=14)
  ax[axr,axc].set_xscale('log')

ax[0,0].legend(loc='upper right',fontsize=14)




It can be seen that in general, the power law slope of the power spectrum of QSO is smaller. 

Now let's see the distribution of the Damp Random Walk (DRW) parameters.  A DRW model is defined by a stochastic differential equation which includes a damping term that pushes the signal back to its mean: $dX(t)=-\frac{1}{\tau_{DRW}}X(t)dt+\sigma_{DRW}\sqrt{dt}\,\epsilon(t)+b\,dt,    \quad      \tau_{DRW},\sigma_{DRW},t>0$.  $\tau_{DRW}$ corresponds to the characteristic time for the time series to become roughly uncorrelated, 
$\sigma_{DRW}^2$ corresponds to the squared amplitude of the variations, and $\epsilon(t)$ is a white noise process with zero mean and variance equal to 1. DRW modelling is typically used to describe light curves of active galactic nuclei ([Kelly et al. 2009](https://ui.adsabs.harvard.edu/abs/2009ApJ...698..895K/abstract)). In this case we obtained the $\sigma_{DRW}^2$ and $\tau_{DRW}$ parameters using Gaussian process regression, with a Ornstein-Uhlenbeck kernel, as in [Graham et al. 2017](https://ui.adsabs.harvard.edu/abs/2017MNRAS.470.4112G/abstract). GP\_DRW\_sigma denote $\sigma_{DRW}$, while GP\_DRW\_tau denote  $\tau_{DRW}$.



From $\sigma_{DRW}^2$ we estimated the asymptotic value of the structure function on long timescales as SF$_\infty = \sqrt{2} \sigma_{DRW}$ ([MacLeod et al. 2011](https://ui.adsabs.harvard.edu/abs/2011ApJ...728...26M/abstract)).





Below we plot the distribution of GP\_DRW\_tau and GP\_DRW\_sigma in the $g$ band, for QSOs, AGNs, and Blazars. 

In [ ]:
feature_x='GP_DRW_sigma_1'
feature_y='GP_DRW_tau_1'

fig, ax = plt.subplots(figsize = (8,8)) 

objects_aux = objects[(objects[feature_x]>=1e-4) & (objects[feature_y]>=1)] # we want to avoid incorrect measurements 

for class_name, color in [("QSO","lightsteelblue"),("AGN","lightcoral"),("Blazar","olivedrab")]:
  plt.scatter(np.log10(np.sqrt(2*objects_aux.loc[objects_aux["class_name"]==class_name,feature_x].values)),np.log10(objects_aux.loc[objects_aux["class_name"]==class_name,feature_y].values),c=color,s=12,label=class_name)


plt.ylabel("log10(%s)" % (feature_y),fontsize=15)
#plt.xlabel("log10(%s)" % (feature_x),fontsize=15)
plt.xlabel(r"log$_{10}(SF_{\infty,g}/mag)$",fontsize=15)
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
plt.legend(fontsize = 15,markerscale=5, loc=2)
plt.ylim(0,4)
plt.xlim(-2.1,0.5)

It can be seen that QSOs, AGNs, and Blazars occupy different locations in this plot, with Blazars showing larger values of $SF_{\infty,g}$ (and therefore GP\_DRW\_sigma\_1). This behavior was also found in a previous analysis presented by [Ruan et al. 2012](https://ui.adsabs.harvard.edu/abs/2012ApJ...760...51R/abstract).

You can see that each object has one row per feature per band. Let's show the unique feature names:

In [ ]:
features.name.unique()

Details on the features can be found in Sánchez-Sáez et al. 2021, AJ, 161, 141

In [ ]:
conn.close()



**Congratulations, you made it to the end of this notebook!**

**If you would like to contribute with your own notebook, please let us know!**
